# SimCLR
PyTorch implementation of SimCLR: A Simple Framework for Contrastive Learning of Visual Representations by T. Chen et al. With support for the LARS (Layer-wise Adaptive Rate Scaling) optimizer.

[Link to paper](https://arxiv.org/pdf/2002.05709.pdf)


## Setup the repository

In [1]:
#!git clone https://github.com/AmeerHamza111/SimCLR.git
%cd SimCLR
#!wget https://github.com/Spijkervet/SimCLR/releases/download/1.2/checkpoint_100.tar
#!sh setup.sh || python3 -m pip install -r requirements.txt || exit 1
#!pip install  pyyaml --upgrade

/content/SimCLR


In [2]:
from google.colab import drive
drive.mount('/data/')
from pathlib import Path
base_dir = ('/data/My Drive')

Drive already mounted at /data/; to attempt to forcibly remount, call drive.mount("/data/", force_remount=True).


In [0]:
#!cp  /data/My\ Drive/DeepLearning/student_data.zip /content/SimCLR

In [0]:
#!unzip student_data.zip

# Part 1:
## SimCLR pre-training

In [0]:
# whether to use a TPU or not (set in Runtime -> Change Runtime Type)
use_tpu = False

#### Install PyTorch/XLA

In [0]:
if use_tpu:
  VERSION = "20200220" #@param ["20200220","nightly", "xrt==1.15.0"]
  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python pytorch-xla-env-setup.py --version $VERSION

In [7]:
import os
import torch

if use_tpu:
  # imports the torch_xla package for TPU support
  import torch_xla
  import torch_xla.core.xla_model as xm
  dev = xm.xla_device()
  print(dev)
  
import torchvision
import argparse

from torch.utils.tensorboard import SummaryWriter

apex = False
try:
    from apex import amp
    apex = True
except ImportError:
    print(
        "Install the apex package from https://www.github.com/nvidia/apex to use fp16 for training"
    )

from model import load_model, save_model
from modules import NT_Xent
from modules.transformations import TransformsSimCLR
from utils import mask_correlated_samples, post_config_hook


Install the apex package from https://www.github.com/nvidia/apex to use fp16 for training


In [0]:
#!pip install sacred

In [0]:
def train(args, train_loader, model, criterion, optimizer, writer):
    loss_epoch = 0
    for step, ((x_i, x_j), _) in enumerate(train_loader):

        optimizer.zero_grad()
        x_i = x_i.to(args.device)
        x_j = x_j.to(args.device)

        # positive pair, with encoding
        h_i, z_i = model(x_i)
        h_j, z_j = model(x_j)

        loss = criterion(z_i, z_j)

        if apex and args.fp16:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()

        optimizer.step()

        if step % 50 == 0:
            print(f"Step [{step}/{len(train_loader)}]\t Loss: {loss.item()}")

        writer.add_scalar("Loss/train_epoch", loss.item(), args.global_step)
        loss_epoch += loss.item()
        args.global_step += 1

    return loss_epoch

### Load arguments from `config/config.yaml`

In [0]:
from pprint import pprint
from utils.yaml_config_hook import yaml_config_hook

config = yaml_config_hook("./config/config.yaml")
args = argparse.Namespace(**config)

if use_tpu:
  args.device = dev
else:
  args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
args.out_dir = "logs"
if not os.path.exists("logs"):
  os.makedirs("logs")

In [11]:
args.batch_size = 2
args.epochs = 1
args.epoch_num = 1
args.resnet = "resnet18"
args.dataset = "road"
args.model_path = "logs/simclrPretext"
#args.optimizer = "LARS"
pprint(vars(args))

{'batch_size': 2,
 'dataset': 'road',
 'device': device(type='cuda', index=0),
 'epoch_num': 1,
 'epochs': 1,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 100,
 'model_path': 'logs/simclrPretext',
 'normalize': True,
 'optimizer': 'Adam',
 'out_dir': 'logs',
 'projection_dim': 64,
 'resnet': 'resnet18',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [0]:
image_folder = 'data'
annotation_csv = 'data/annotation.csv'

import numpy as np
from data_helper import SimclrUnlabeledDataset
from helper import convert_map_to_lane_map, convert_map_to_road_map, collate_fn, draw_box

unlabeled_scene_index = np.arange(106)

### Load dataset into train loader

In [0]:
root = "./datasets"

train_sampler = None

if args.dataset == "STL10":
    train_dataset = torchvision.datasets.STL10(
        root, split="unlabeled", download=True, transform=TransformsSimCLR()
    )
elif args.dataset == "CIFAR10":
    train_dataset = torchvision.datasets.CIFAR10(
        root, download=True, transform=TransformsSimCLR()
    )
elif args.dataset == "road":
    train_dataset = SimclrUnlabeledDataset(image_folder=image_folder, 
      scene_index=unlabeled_scene_index, first_dim='sample', transform=TransformsSimCLR())
else:
    raise NotImplementedError

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    shuffle=(train_sampler is None),
    drop_last=True,
    num_workers=16,
    sampler=train_sampler,
)

### Load the SimCLR model, optimizer and learning rate scheduler

In [0]:
model, optimizer, scheduler = load_model(args, train_loader)

In [15]:
print(model)

SimCLR(
  (encoder): ResNet(
    (conv1): Conv2d(18, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

### Setup TensorBoard for logging experiments

In [0]:
tb_dir = os.path.join(args.out_dir, "colab")
if not os.path.exists(tb_dir):
  os.makedirs(tb_dir)
writer = SummaryWriter(log_dir=tb_dir)

### Create the mask that will remove correlated samples from the negative examples

In [0]:
mask = mask_correlated_samples(args)

### Initialize the criterion (NT-Xent loss)

In [0]:
criterion = NT_Xent(args.batch_size, args.temperature, mask, args.device)

### Start training

In [19]:
'''args.global_step = 0
args.current_epoch = 0
for epoch in range(args.start_epoch, args.epochs):
    lr = optimizer.param_groups[0]['lr']
    loss_epoch = train(args, train_loader, model, criterion, optimizer, writer)

    if scheduler:
        scheduler.step()

    if epoch % 5 == 0:
        save_model(args, model, optimizer)

    writer.add_scalar("Loss/train", loss_epoch / len(train_loader), epoch)
    writer.add_scalar("Misc/learning_rate", lr, epoch)
    print(
        f"Epoch [{epoch}/{args.epochs}]\t Loss: {loss_epoch / len(train_loader)}\t lr: {round(lr, 5)}"
    )
    args.current_epoch += 1

## end training
save_model(args, model, optimizer)'''

'args.global_step = 0\nargs.current_epoch = 0\nfor epoch in range(args.start_epoch, args.epochs):\n    lr = optimizer.param_groups[0][\'lr\']\n    loss_epoch = train(args, train_loader, model, criterion, optimizer, writer)\n\n    if scheduler:\n        scheduler.step()\n\n    if epoch % 5 == 0:\n        save_model(args, model, optimizer)\n\n    writer.add_scalar("Loss/train", loss_epoch / len(train_loader), epoch)\n    writer.add_scalar("Misc/learning_rate", lr, epoch)\n    print(\n        f"Epoch [{epoch}/{args.epochs}]\t Loss: {loss_epoch / len(train_loader)}\t lr: {round(lr, 5)}"\n    )\n    args.current_epoch += 1\n\n## end training\nsave_model(args, model, optimizer)'

## Download last checkpoint to local drive (replace `100` with `args.epochs`)

In [0]:
#from google.colab import files
#files.download('./logs/checkpoint_100.tar')

In [0]:
def compute_bb_loss(boxes1, boxes2):
    boxes1 = convertBoundingBoxes(boxes1)
    boxes2 = convertBoundingBoxes(boxes2)
    
    num_boxes1 = boxes1.size(0)
    num_boxes2 = boxes2.size(0)

    boxes1_max_x = boxes1[:, 0].max(dim=1)[0]
    boxes1_min_x = boxes1[:, 0].min(dim=1)[0]
    boxes1_max_y = boxes1[:, 1].max(dim=1)[0]
    boxes1_min_y = boxes1[:, 1].min(dim=1)[0]

    boxes2_max_x = boxes2[:, 0].max(dim=1)[0]
    boxes2_min_x = boxes2[:, 0].min(dim=1)[0]
    boxes2_max_y = boxes2[:, 1].max(dim=1)[0]
    boxes2_min_y = boxes2[:, 1].min(dim=1)[0]

    condition1_matrix = (boxes1_max_x.unsqueeze(1) > boxes2_min_x.unsqueeze(0))
    condition2_matrix = (boxes1_min_x.unsqueeze(1) < boxes2_max_x.unsqueeze(0))
    condition3_matrix = (boxes1_max_y.unsqueeze(1) > boxes2_min_y.unsqueeze(0))
    condition4_matrix = (boxes1_min_y.unsqueeze(1) < boxes2_max_y.unsqueeze(0))
    condition_matrix = condition1_matrix * condition2_matrix * condition3_matrix * condition4_matrix

    iou_matrix = torch.zeros(num_boxes1, num_boxes2)
    k = 0
    loss = 0
    for i in range(num_boxes1):
        for j in range(num_boxes2):
            if condition_matrix[i][j]:
                iou_matrix[i][j] = compute_iou(boxes1[i], boxes2[j])
                loss += - torch.log(iou_matrix[i][[j]])
                k += 1
    return loss/k

def compute_iou(box1, box2):
    a = Polygon(torch.t(box1)).convex_hull
    b = Polygon(torch.t(box2)).convex_hull
    
    return a.intersection(b).area / a.union(b).area

def convertBoundingBoxes(boxes):
    convBoxes = []
    for box in boxes: 
        xmin = box[0]
        ymin = box[1]
        xmax = box[2]
        ymax = box[3]
        cbox = [[xmin,xmin,xmax,xmax], [ymin,ymax,ymin,ymax]]
        convBoxes.append(cbox)
        convBoxes = torch.Tensor(convBoxes)
        return convBoxes

In [22]:
from modelobj import NetObj
mdl = NetObj()
model1 = mdl.model
model1.to(args.device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

In [0]:
import torch.nn as nn

model, _, _ = load_model(args, train_loader, reload_model=True)

In [0]:
model2 = nn.Sequential(model.encoder.conv1, model.encoder.bn1,model.encoder.relu, model.encoder.maxpool,
                      model.encoder.layer1, model.encoder.layer2, model.encoder.layer3, model.encoder.layer4[0])

model3 = nn.Sequential(model.encoder.layer4[1].conv1,model.encoder.layer4[1].bn1, nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),                     
nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),model1.backbone.fpn.extra_blocks,model1.rpn, model1.roi_heads)

In [25]:
model2

Sequential(
  (0): Conv2d(18, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Co

In [26]:
model3

Sequential(
  (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LastLevelMaxPool()
  (5): RegionProposalNetwork(
    (anchor_generator): AnchorGenerator()
    (head): RPNHead(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cls_logits): Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
      (bbox_pred): Conv2d(256, 12, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (6): RoIHeads(
    (box_roi_pool): MultiScaleRoIAlign()
    (box_head): TwoMLPHead(
      (fc6): Linear(in_features=12544, out_features=1024, bias=True)
      (fc7): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (box_predictor): FastRCNNPredictor(
      (cls_score):

In [0]:
#from engine import train_one_epoch, evaluate
#import utils

# Part 2:
## Linear evaluation using logistic regression, using weights from frozen, pre-trained SimCLR model

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import argparse

from experiment import ex
from model import load_model
from utils import post_config_hook

#from modules import LogisticRegression


In [0]:
def train(args, loader, model2, model3, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    model2.to(args.device)
    model3.to(args.device)
    model2.eval()
    model3.train()
    for step, (x, y) in enumerate(loader):
        optimizer.zero_grad()

        #print(x)
        print(y)
        #print(y.shape)

        x = x.to(args.device)
        y = y.to(args.device)

        # get encoding
        with torch.no_grad():
            h = model2(x)
            print(h.shape)
            # h = 512
            # z = 64

        output = model3(h[0],y)
        print(output)
        loss = compute_bb_loss(output['boxes'], y)

        '''predicted = output.argmax(1)
        acc = (predicted == y).sum().item() / y.size(0)
        accuracy_epoch += acc'''

        loss.backward()
        optimizer.step()

        loss_epoch += loss.item()
        if step % 1 == 0:
            print(f"Step [{step}/{len(loader)}]\t Loss: {loss.item()}\t Accuracy: 0")

    return loss_epoch, 0

In [0]:
def test(args, loader, model2, model3, model, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    model2.to(args.device)
    model3.to(args.device)
    model2.eval()
    model3.eval()
    for step, (x, y) in enumerate(loader):
        model.zero_grad()

        x = x.to(args.device)
        y = y.to(args.device)

        # get encoding
        with torch.no_grad():
            h = model2(x)
            # h = 512
            # z = 64

        output = model3(h)
        loss = compute_bb_loss(output['boxes'], y)

        '''predicted = output.argmax(1)
        acc = (predicted == y).sum().item() / y.size(0)
        accuracy_epoch += acc'''

        loss_epoch += loss.item()


    return loss_epoch, 0

In [31]:
from pprint import pprint
from utils.yaml_config_hook import yaml_config_hook

config = yaml_config_hook("./config/config.yaml")
pprint(config)
args = argparse.Namespace(**config)

if use_tpu:
  args.device = dev
else:
  args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

{'batch_size': 128,
 'dataset': 'STL10',
 'epoch_num': 100,
 'epochs': 100,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 100,
 'model_path': 'logs/0',
 'normalize': True,
 'optimizer': 'Adam',
 'projection_dim': 64,
 'resnet': 'resnet50',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [32]:
args.batch_size = 2
args.resnet = "resnet18"
args.model_path = "logs/simclrPretext"
args.epochs = 1
args.epoch_num = 1
args.dataset = 'road'
args.logistic_epochs =1
args.logistic_batch_size = 1
args.out_dir = "logs/train"
#args.optimizer = "LARS"
pprint(vars(args))

{'batch_size': 2,
 'dataset': 'road',
 'device': device(type='cuda', index=0),
 'epoch_num': 1,
 'epochs': 1,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 1,
 'logistic_epochs': 1,
 'model_path': 'logs/simclrPretext',
 'normalize': True,
 'optimizer': 'Adam',
 'out_dir': 'logs/train',
 'projection_dim': 64,
 'resnet': 'resnet18',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [0]:
labeled_scene_index = np.arange(106, 120)

## validation scene index.
validation_scene_index = np.arange(120, 134)

In [0]:
train_transform = torchvision.transforms.Compose(
            [
                #torchvision.transforms.RandomResizedCrop(size=(256,306)),
                torchvision.transforms.ToTensor(),
            ]
        )

In [0]:
from data_helper import SimclrLabeledBBDataset, SimclrLabeledDataset, ObjDetectionLabeledDataset

In [0]:
from helper import collate_fn, draw_box

### Load dataset into train/test dataloaders

In [0]:
root = "./datasets"
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

if args.dataset == "STL10":
    train_dataset = torchvision.datasets.STL10(
        root, split="train", download=True, transform=torchvision.transforms.ToTensor()
    )
    test_dataset = torchvision.datasets.STL10(
        root, split="test", download=True, transform=torchvision.transforms.ToTensor()
    )
elif args.dataset == "CIFAR10":
    train_dataset = torchvision.datasets.CIFAR10(
        root, train=True, download=True, transform=transform
    )
    test_dataset = torchvision.datasets.CIFAR10(
        root, train=False, download=True, transform=transform
    )
elif args.dataset == "road":
    train_dataset =  SimclrLabeledBBDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=labeled_scene_index,
                                  transform=transform,
                                  extra_info=False
                                 )
    test_dataset = SimclrLabeledBBDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=validation_scene_index,
                                  transform=transform,
                                  extra_info=False
                                 )
else:
    raise NotImplementedError

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.logistic_batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=args.logistic_batch_size,
    shuffle=False,
    drop_last=True,
    num_workers=4
)

In [0]:
#a,b = iter(train_loader).next()
#print(a)
#print(b)

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [40]:
'''for epoch in range(0, 5):
  # train for one epoch, printing every 10 iterations
  train_one_epoch(model2, model3, optimizer, train_loader, args.device, epoch, print_freq=10)
  # update the learning rate
  lr_scheduler.step()
  # evaluate on the test dataset
  evaluate(model2, model3, test_loader, device=args.device)

  model_file = 'modobj/fasterrcnn_model_' + str(epoch) + '.pth'
  torch.save({'modelObjectDetection_state_dict': model.state_dict()},model_file)
  print('\nSaved model to ' + model_file )'''

"for epoch in range(0, 5):\n  # train for one epoch, printing every 10 iterations\n  train_one_epoch(model2, model3, optimizer, train_loader, args.device, epoch, print_freq=10)\n  # update the learning rate\n  lr_scheduler.step()\n  # evaluate on the test dataset\n  evaluate(model2, model3, test_loader, device=args.device)\n\n  model_file = 'modobj/fasterrcnn_model_' + str(epoch) + '.pth'\n  torch.save({'modelObjectDetection_state_dict': model.state_dict()},model_file)\n  print('\nSaved model to ' + model_file )"

### Load SimCLR model and load model weights

In [41]:
'''simclr_model, _, _ = load_model(args, train_loader, reload_model=True)
simclr_model = simclr_model.to(args.device)
simclr_model.eval()'''

'simclr_model, _, _ = load_model(args, train_loader, reload_model=True)\nsimclr_model = simclr_model.to(args.device)\nsimclr_model.eval()'

In [42]:
## Logistic Regression
''''n_classes = 9 # stl-10
model = LogisticRegression(simclr_model.n_features, n_classes)
model = model.to(args.device)'''

"'n_classes = 9 # stl-10\nmodel = LogisticRegression(simclr_model.n_features, n_classes)\nmodel = model.to(args.device)"

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()

In [44]:
for epoch in range(args.logistic_epochs):
    loss_epoch, accuracy_epoch = train(args, train_loader, model2, model3, criterion, optimizer)
    print(f"Epoch [{epoch}/{args.logistic_epochs}]\t Loss: {loss_epoch / len(train_loader)}\t Accuracy: {accuracy_epoch / len(train_loader)}")

    # final testing
    loss_epoch, accuracy_epoch = test(args, test_loader, model2, model3, criterion, optimizer)
    print(f"[FINAL]\t Loss: {loss_epoch / len(test_loader)}\t Accuracy: {accuracy_epoch / len(test_loader)}")

{'boxes': tensor([[[591.7317, 354.3498, 631.2488, 373.1270],
         [341.1134, 155.0004, 391.7066, 176.0125],
         [546.3565, 497.3262, 585.4752, 514.8645],
         [617.9739, 383.6549, 709.0997, 414.2684],
         [ 99.9079, 389.4792, 147.2225, 409.4612],
         [341.1930, 354.6389, 388.4191, 372.9422],
         [597.5522, 496.1530, 648.1851, 515.6973],
         [118.3289, 610.1826, 138.0472, 655.3387]]]), 'category': tensor([[2, 2, 2, 5, 2, 2, 2, 2]])}


AttributeError: ignored